# Interactive visualization

In this homework we will practice with interactive visualization. 
We will be working with the P3 database of the SNSF [(Swiss National Science Foundation)](http://www.snf.ch/en/Pages/default.aspx). and build a Choropleth map which shows intuitively  how much grant money goes to each Swiss canton.

In [1]:
import folium
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt

We decide to not use Geomap nor Google Maps API because after some tries there are still some work to do by hand. We will get the data manually.

## Data Acquisition
Here, we do a first parsing and put the data in the dataFrame **grant_data**. We keep the rows where the canton of funding is clearly indicated.
We drop the rows for which we won't be able to extract the canton of funding or where the grant ammount is not specified and put them in an other dataFrame **grant_data_missing**. The latter will be investigated later.

In [2]:
# Loading the P3 grant data
swiss_grant = r'P3_GrantExport.csv'
grant_data = pd.read_csv(swiss_grant,  sep=';', header=0, index_col=0, usecols=[0,6,7,13])

#Keep the rows where the canton is clearly indicated in the University column in the original dataframe
#moving the others to a new dataframe _missing
grant_data_missing = grant_data[grant_data.University.isnull() | grant_data.University.isin(["Nicht zuteilbar - NA"])]
grant_data = grant_data.drop(grant_data_missing.index)

#dropping the rows for which we won't be able to extract the canton of funding
grant_data_missing = grant_data_missing[((grant_data_missing.University.isin(["Nicht zuteilbar - NA"]) | grant_data_missing.University.isnull() )& grant_data_missing.Institution.isnull()  ) == False]
#dropping rows where grant amount is not specified
grant_data_missing.drop(grant_data_missing.ix[grant_data_missing['Approved Amount'] == "data not included in P3"].index, inplace= True)


### Requirements on the data size

We have a total of 5211 different Institutions registered in the data. We aim for 95% geolocalisation, that is we should have more than 4950 parsed Institutions.
We aso have a total of 76 different universities. We aim for 95% geolocalisation, that is we should have more than 72 parsed universities.

In [3]:
print(grant_data.Institution.unique().size)
print(grant_data.University.unique().size)
grant_data.count()

5211
76


Institution        47157
University         48393
Approved Amount    48393
dtype: int64

In the column "University", we notice that most of the time the canton is indicated as follow :

- name of the university - canton.

Hence, for some rows, it allows to get the canton easily. 

However, there are some universities that don't display the canton, as "EPFL". Hence, we manually assign them to their respective canton.

Some universities, as the "HES-SO" are located in several cantons. We put them in an other dataFrame, called **grant_data_refine**

In [4]:
# get the canton from the university name.
grant_data['Canton']= grant_data["University"].str.split("- ").str[1]

# Manually assign universities to their respective canton
grant_data.loc[grant_data.Canton.isin(['LA', 'EPFL', 'HEPL', 'EHB', 'FORS', 'IST', 'SIB' ]), 'Canton'] = 'VD'
grant_data.loc[grant_data.Canton.isin(['PHBern', 'RWS', 'BFH']), 'Canton'] = 'BE'
grant_data.loc[grant_data.Canton.isin(['ETHZ', 'PHZFH', 'SIK-ISEA', 'EAWAG', 'EMPA', 'FHKD', 'HfH', 'ZFH']), 'Canton'] = 'ZH'
grant_data.loc[grant_data.Canton.isin(['IHEID']), 'Canton'] = 'GE'
grant_data.loc[grant_data.Canton.isin(['PHGR', 'PMOD', 'AORI', "und Asthmaforschung ", 'IKG']), 'Canton'] = 'GR'
grant_data.loc[grant_data.Canton.isin(['SUPSI', 'IRSOL', 'FUS', 'EOC', 'USI']), 'Canton'] = 'TI'
grant_data.loc[grant_data.Canton.isin(['PSI', 'PHFHNW', 'FIBL','FHNW']), 'Canton'] = 'AG'
grant_data.loc[grant_data.Canton.isin(['PHSG', 'SHLR', 'KSPSG', 'FHO']), 'Canton'] = 'SG'
grant_data.loc[grant_data.Canton.isin(['CSEM', 'ISSKA']), 'Canton'] = 'NE'
grant_data.loc[grant_data.Canton.isin(['HEPFR']), 'Canton'] = 'FR'
grant_data.loc[grant_data.Canton.isin(['FFHS', 'PHVS', 'CREALP', 'IDIAP', 'IUKB', 'IRO']), 'Canton'] = 'VS'
grant_data.loc[grant_data.Canton.isin(['STHB', 'FMI']), 'Canton'] = 'BS'
grant_data.loc[grant_data.Canton.isin(['PHSH']), 'Canton'] = 'SH'
grant_data.loc[grant_data.Canton.isin(['SPF', 'PHLU', 'HSLU']), 'Canton'] = 'LU'
grant_data.loc[grant_data.Canton.isin(['PHZG']), 'Canton'] = 'ZG'
grant_data.loc[grant_data.Canton.isin(['BITG', 'PHTG']), 'Canton'] = 'TG'
grant_data.loc[grant_data.Canton.isin(['PHSZ']), 'Canton'] = 'SZ'
grant_data.loc[grant_data.Canton.isin(['HEPBEJUNE']), 'Canton'] = 'JU'

#Put rows for which we found information in a new dataFrame : grant_data_refine
grant_data_refine =grant_data[grant_data.Canton.isin(['NPO', 'HES-SO','ASPIT','FINST','WSL','FP','FTL','IST','ASP', 'AGS'])]

#Delete rows for which we found information from grant_data
grant_data.drop(grant_data.ix[grant_data.Canton.isin(['NPO','HES-SO','ASPIT','FINST','WSL','FP','FTL','ASP', 'AGS'])].index, inplace= True)

#NPO, HES-SO, ASPIT, FINST,WSL, FP, FTL, ASP, AGS need further parsing
#ISO is in Roma -> delete

grant_data_refine = pd.concat([grant_data_refine,grant_data_missing])

In [5]:
print(grant_data.Institution.unique().size)
print(grant_data.University.unique().size)
grant_data.count()

4509
67


Institution        44536
University         45671
Approved Amount    45671
Canton             45670
dtype: int64

Now we are going to analyze further the rows that were harder to parse. These entries were put in the dataFrame
**grant_data_refine**

We parse manually the Institution using the city name written in Institution or the name of the institution and keywords.

Finally, we put the entries that we have been able to find in a new dataFrame : **grant_data_refined**. We drop these entries in the **grant_data_refine**.

In [6]:
#Find keyword in the institution name
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Genève') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Geneva') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('travail social', case=0) == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('HEAD') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Droz') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('CERN') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Martin Bodmer') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Muséum d'Histoire Naturelle") == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('genevoise') == True, 'Canton'] = 'GE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Basel') == True, 'Canton'] = 'BS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Valais') == True, 'Canton'] = 'VS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('CREPA') == True, 'Canton'] = 'VS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Sion') == True, 'Canton'] = 'VS'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Fribourg') == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Ecole d'ingénieurs et d'architectes") == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Forschungskomitee Rechtssoziologie") == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('fribourgeoise') == True, 'Canton'] = 'FR'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Zür') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Eawag') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('ART') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Winterthur') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('ETH') == True, 'Canton'] = 'ZH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Arc', case=0) == True, 'Canton'] = 'JU'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Schaffhausen', case=0) == True, 'Canton'] = 'SH'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Bibliothèque cantonale jurassienne', case=0) == True, 'Canton'] = 'JU'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Lausanne') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Bibliothèque cantonale') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Vaud', case=0) == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('EPFL') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('EPF-L') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Gonin') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Suisse Occidental') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Lullier') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('La Source') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Changins') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains("Ecole d'études sociales et pédagogiques") == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('HECV') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Heig-VD', case=0) == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('CHUV') == True, 'Canton'] = 'VD'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Bern') == True, 'Canton'] = 'BE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Schweizerische Musikforschende Gesellschaft') == True, 'Canton'] = 'BE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Neuchâtel') == True, 'Canton'] = 'NE'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Scherrer') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Hürlimann') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Baden') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Aar') == True, 'Canton'] = 'AG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('cantonale', case=0) == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Ticin', case=0) == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Lugano') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Liceo') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Svizzera') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Sezione') == True, 'Canton'] = 'TI'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Luzern') == True, 'Canton'] = 'LU'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Thurgau') == True, 'Canton'] = 'TG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('SLF') == True, 'Canton'] = 'GR'  
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Rumants') == True, 'Canton'] = 'GR' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Schwyz') == True, 'Canton'] = 'SZ'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('St. Gall') == True, 'Canton'] = 'SG'
grant_data_refine.loc[grant_data_refine.Institution.str.contains('St.Gall') == True, 'Canton'] = 'SG' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Zug') == True, 'Canton'] = 'ZG' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Solothurn') == True, 'Canton'] = 'SO' 
grant_data_refine.loc[grant_data_refine.Institution.str.contains('Appenzell') == True, 'Canton'] = 'AR' 

#Put rows for which we found information in a new dataFrame : grant_data_refined
grant_data_refined = grant_data_refine[grant_data_refine.Canton.isin(['GE','SZ','SO','AG', 'TI', 'ZG', 'GR','SG', 'BS','VS', 'LU', 'FR','ZH', 'LA', 'BE', 'NE', 'AR', 'JU', 'TG'])]

#Delete rows for which we found information from grant_data_refine
grant_data_refine.drop(grant_data_refine.ix[grant_data_refine.Canton.isin(['ZG','SZ','SO','AG', 'TI', 'GE','GR','SG', 'BS','LU', 'TG', 'VS','FR','ZH', 'LA', 'BE', 'NE', 'AR', 'JU'])].index, inplace= True)

# Some universities or instituions are outside of switzerland. They have some keywords in their title. Hence we can drop them
grant_data_refine.drop(grant_data_refine.ix[grant_data_refine.University.isnull()].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('Institute', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('Department', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('University', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('Universität', case=0) == True].index, inplace= True)
grant_data_refine.drop(grant_data_refine.loc[grant_data_refine.Institution.str.contains('UNI', case=0) == True].index, inplace= True)

#Concatenate grant_data and grant_data_refined
grant_data = pd.concat([grant_data,grant_data_refined])
grant_data.dropna(subset=['Canton'],inplace=True)
grant_data.drop(grant_data.ix[grant_data.Canton.isin(['NPO','HES-SO','ASPIT','FINST','WSL','FP','FTL','ASP', 'AGS', 'ISR'])].index, inplace= True)

### Final Data

With the manual work, we have been able to get 75 different universities and 5037 different instituions.

In [7]:
print(grant_data.Institution.unique().size)
print(grant_data.University.unique().size)
grant_data.count()

5037
75


Approved Amount    47322
Canton             47322
Institution        46187
University         46685
dtype: int64

**Na values**
However, when acquiring more entries, we got some NA values. We will delete them.

In [8]:
grant_data.loc[grant_data['Approved Amount'].isin(['data not included in P3'])].head(5)

,Approved Amount,Canton,Institution,University
"﻿""Project Number""",,,,
65748,data not included in P3,NE,Institut de Zoologie Faculté des Sciences Univ...,Université de Neuchâtel - NE
65755,data not included in P3,BE,Geographisches Institut Universität Bern,Universität Bern - BE
66155,data not included in P3,ZH,Biochemisches Institut Universität Zürich,Universität Zürich - ZH
66164,data not included in P3,VD,Institut Suisse de Recherche Expérimentale sur...,EPF Lausanne - EPFL
66237,data not included in P3,BS,Departement Physik Universität Basel,Universität Basel - BS


In [9]:
#dropping rows where grant amount is not specified
grant_data.drop(grant_data.ix[grant_data['Approved Amount'] == "data not included in P3"].index, inplace= True)
grant_data.dropna(subset=['Approved Amount'])  

#checking NA
print(grant_data.loc[grant_data['Approved Amount'].isin(['data not included in P3'])].sum())
print(grant_data['Approved Amount'].isnull().sum())

#Checking size
print(grant_data.Institution.unique().size)
print(grant_data.University.unique().size)
grant_data.count()

Approved Amount    0.0
Canton             0.0
Institution        0.0
University         0.0
dtype: float64
0
5031
75


Approved Amount    47212
Canton             47212
Institution        46078
University         46575
dtype: int64

## Visualization Preparation

In the dataFrame **grant_canton** we compute the sum of the approved ammount by canton.
For some canton, the ammount is 0

In [10]:
#Compute the sums
grant_data['Approved Amount'] = pd.to_numeric(grant_data['Approved Amount'], errors='coerce')
grant_canton = grant_data.groupby(grant_data.Canton).sum()
grant_canton.loc['UR'] = 0.0
grant_canton.loc['NW'] = 0.0
grant_canton.loc['OW'] = 0.0
grant_canton.loc['GL'] = 0.0
grant_canton.loc['BL'] = 0.0
grant_canton.loc['AI'] = 0.0
grant_canton['cantons'] = grant_canton.index
grant_canton.sort_values('Approved Amount', ascending = False)

,Approved Amount,cantons
Canton,,
ZH,3.695114e+09,ZH
VD,2.415326e+09,VD
GE,1.901525e+09,GE
BE,1.581336e+09,BE
BS,1.398939e+09,BS
FR,4.614244e+08,FR
NE,4.050351e+08,NE
AG,1.798424e+08,AG
TI,1.247166e+08,TI


## Visualization

In [11]:
#Build the folium map of Switzerland with highlighted cantons

map = folium.Map(location=[46.8,8],  tiles='Mapbox Bright', zoom_start=8)
swiss_topo = r'ch-cantons.topojson.json'

map.choropleth(geo_path = swiss_topo, topojson = 'objects.cantons',
               data=grant_canton,
               columns=['cantons', 'Approved Amount'],
               key_on='feature.id',
               fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5,
               legend_name='Grant Amount',
               threshold_scale = [ 1.7e+05, 3.6e+07, 1.0e+08, 1.3e+09, 2.4e+09, 3.69e+09],
               reset=True)
map

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 905: ordinal not in range(128)

# Bonus

Switzerland is culturaly split by the Röstigraben. Let's look at the differences in funding between the areas.

We decide to manually assign a language value to each canton. There are 3 categories of cantons: 
- Those in which people speak French
- Those in which people speak German
- Those in which people speak Germand and French

We will not analyze the canons where people speak Italian, the entries will be dropped.

In [107]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/9/9f/Map_Languages_CH.png")

**French speaking Universities**

In [108]:
french_university = grant_data[grant_data.Canton.isin(['GE','JU', 'VD','NE'])]
french_university = french_university.copy()
french_university['Language']='FR'

**German speaking Universities**

In [109]:
german_university = grant_data[grant_data.Canton.isin(['SZ','SO','AG', 'ZG', 'GR','SG', 'BS', 'LU','ZH', 'LA', 'BE', 'AR', 'TG'])]
german_university = german_university.copy()
german_university['Language']='DE'

### Other Universities 

#### Where to cut the Rostigraben ?
Valais, Fribourg and Bern are the 3 cantons that are cut by the Rostigraben. However, depending on where the university is in the canton, it might be in the french or german speaking part.
Let's have a look at each of these university for each canton. We will look for the name of the university on google, and depending on the language spoken in the university's city, we will assign a language.

#### Valais 
In valais, all the univeristies are in the French speaking part, besides :

- 'Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS'
- 'Pädagogische Hochschule Wallis - PHVS'

which are located in Brig, in the Swiss German part

In [110]:
a = grant_data[grant_data.Canton.isin(['VS'])]
a['University'].unique()

array(["Centre de rech. sur l'environnement alpin - CREALP",
       'Idiap Research Institute - IDIAP',
       'Institut Universitaire Kurt Bösch - IUKB',
       'Forschungsinstitut für Opthalmologie - IRO',
       'Pädagogische Hochschule Wallis - PHVS',
       'Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS',
       'NPO (Biblioth., Museen, Verwalt.) - NPO',
       'Weitere Spitäler - ASPIT', 'HES de Suisse occidentale - HES-SO'], dtype=object)

In [111]:
valais_university = grant_data[grant_data.Canton.isin(['VS'])]
valais_university = valais_university.copy()
valais_university['Language'] = 'FR'
valais_university.loc[valais_university['University'].str.contains('Fernfachhochschule Schweiz') == True, 'Language'] = 'DE' 
valais_university.loc[valais_university['Institution'].str.contains('Fernfachhochschule Schweiz') == True, 'Language'] = 'DE' 
valais_university.loc[valais_university['University'].str.contains('Pädagogische Hochschule Wallis - PHVS') == True, 'Language'] = 'DE' 
valais_university.loc[valais_university['Institution'].str.contains('Pädagogische Hochschule Wallis - PHVS') == True, 'Language'] = 'DE' 

#valais_university[valais_university['University'].isin(['Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS'])]

#### Fribourg

**Assumption** We say that the city of Fribourg is french speaking as 80% of the population speak French


In Fribourg, all the universities are in the city of Fribourg ( French speaking) , but :

- 'Firmen/Privatwirtschaft - FP'

which is located in Fribourg, but speaks German.

In [112]:
a = grant_data[grant_data.Canton.isin(['FR'])]
a['University'].unique()

array(['Université de Fribourg - FR',
       'Haute école pédagogique fribourgeoise - HEPFR',
       'NPO (Biblioth., Museen, Verwalt.) - NPO',
       'Firmen/Privatwirtschaft - FP',
       'HES de Suisse occidentale - HES-SO', nan], dtype=object)

In [113]:
fribourg_university = grant_data[grant_data.Canton.isin(['FR'])]
fribourg_university = fribourg_university.copy()
fribourg_university['Language'] = 'FR'
fribourg_university.loc[fribourg_university['University'].str.contains('Firmen/Privatwirtschaft') == True, 'Language'] = 'DE' 
fribourg_university.loc[fribourg_university['Institution'].str.contains('Firmen/Privatwirtschaft') == True, 'Language'] = 'DE' 

#### Bern

**Assumption** We say that the city of Bern is German speaking.


In Bern, all the univeristies are in the city of Bern ( German speaking)..

In [114]:
a = grant_data[grant_data.Canton.isin(['BE'])]
a['University'].unique()

array(['Universität Bern - BE', 'Robert Walser-Stiftung Bern - RWS',
       'Berner Fachhochschule - BFH',
       'Pädagogische Hochschule Bern - PHBern',
       'NPO (Biblioth., Museen, Verwalt.) - NPO',
       'Firmen/Privatwirtschaft - FP',
       'Forschungsanstalten Agroscope - AGS', 'Weitere Institute - FINST',
       nan], dtype=object)

In [115]:
bern_university = grant_data[grant_data.Canton.isin(['BE'])]
bern_university = bern_university.copy()
bern_university['Language'] = 'DE'

#### Concatenate all the dataFrames

In [116]:
language_cantons = pd.concat([french_university,german_university,valais_university,fribourg_university,bern_university])

Checking for NA values

In [118]:
#Checking NaN
print(language_cantons[language_cantons['Approved Amount'].isnull()==True].sum())

Approved Amount    0.0
Canton             0.0
Institution        0.0
University         0.0
Language           0.0
dtype: float64


In the dataFrame **language_cantons** we compute the sum of the approved ammount by canton.

In [119]:
#Compute the sums
language_cantons['Approved Amount'] = pd.to_numeric(language_cantons['Approved Amount'], errors='coerce')
language_cantons1 = language_cantons.groupby(language_cantons.Language).sum()
language_cantons1.sort_values('Approved Amount', ascending = False)

,Approved Amount
Language,
DE,8.658893e+09
FR,5.272078e+09


Calculating the difference

In [120]:
language_cantons1['Approved Amount'][0]-language_cantons1['Approved Amount'][1]

3386814548.3699875

The german part recieved 3.5 millards CHF of funding more than the french part since 1975

#### Funding by university

But we have somehow to take into account that the german part is bigger than the french part. We will count the number of universities in each part of Rostigraben, and calculates in average how much each university recieved during these last 40 years

In [149]:
#Calculating how many universities there are in each part
df=language_cantons[['University','Language','Canton']]
df=df.drop_duplicates()
del df['Canton']
number_of_universities=df.groupby(df.Language).count()
number_of_universities

,University
Language,
DE,71
FR,40


Funding by university in the german part

In [150]:
g=language_cantons1['Approved Amount'][0]/number_of_universities['University'][0]
g


121956233.21028158

Funding by university in the german partin the french part

In [151]:
f=language_cantons1['Approved Amount'][1]/number_of_universities['University'][1]
f

131801950.23900013

Difference in funding by university in the french part and german part

In [152]:
f-g

9845717.028718546

In average a University in the French speaking side of the Rostigraben recieved almost 10 millions CHF more than a University in the German speaking part in the last 40 years

#### Funding by institution


In [155]:
#French speaking cantons approved ammount
french_cantons_ammount = language_cantons[language_cantons.Language.isin(['FR'])]['Approved Amount']
german_cantons_ammount = language_cantons[language_cantons.Language.isin(['DE'])]['Approved Amount']

In [156]:
french_cantons_ammount.describe()

count    1.973400e+04
mean     2.671571e+05
std      3.510291e+05
min      0.000000e+00
25%      8.572250e+04
50%      1.845000e+05
75%      3.327505e+05
max      1.548775e+07
Name: Approved Amount, dtype: float64

In [157]:
german_cantons_ammount.describe()

count    3.263800e+04
mean     2.653010e+05
std      3.097191e+05
min      0.000000e+00
25%      8.842000e+04
50%      1.910900e+05
75%      3.343882e+05
max      1.052800e+07
Name: Approved Amount, dtype: float64

#### Statistical tests
"For the funding of the institutions, is the difference in average statistically significant between french speaking and german speaking cantons?"

In [159]:
stats.ttest_ind(french_cantons_ammount, german_cantons_ammount, equal_var = False)

Ttest_indResult(statistic=0.61250893542256835, pvalue=0.54020478826771923)

However the difference in mean is not statistically significant.